# Coleta e preparação dos dados referentes aos roubos de aparelhos celular em São Paulo no primeiro semestre de 2023

## Coleta

Os dados que usaremos neste estudos foram coletados do site da Secretaria de Segurança Publica do governo de São Paulo. São dados públicos disponíveis em https://www.ssp.sp.gov.br/transparenciassp/Consulta.aspx. 

O site apresenta um conjunto de temas cujos quais podemos pesquisar por dados do nosso interesse. Ao selecionar um tema, no nosso caso, "ROUBOS DE CELULAR",  uma tabela de dados com segmentada pelos anos e seus respectivos meses é apresentada. Logo abaixo da tabela, temos dois botões: O primero permite que baixemos a "METODOLOGIA", um arquivo de texto contendo informações sobre os dados e um dicionário de dados. O segundo exporta os dados em um formato `xls` (Excel).

Estou realizando a coleta em 08/2023, e destacaria três pontos de atenção para esta etapa:

1. O download dos dados demoram um tempo relativamente alto. Alguns downloads levaram mais de 3 minutos sem nenhum outro recurso utilizando uma banda de 200 MB.
2. O servidor cai com frequência. Tive que recarregar a página várias vezes pois um erro 500 ocorria toda vez que tentava fazer downloads consecutivos. Por esta razão, por enquanto, preferi não programar um crawler para obter os dados. 
3. Os dados vêm com um nome de arquivo inadequado: `DadosBO_2023_1(ROUBO DE CELULAR)`. Como o uso de parenteses espaços podem causar problemas, teremos que padronizar adequadamente o nome dos arquivos.

## Preparação dos dados 

### Importação dos dados

Uma vez coletados os dados, a importação dos dados apresenta problemas:

In [1]:
import pandas as pd

Se tentarmos importar um arquivo para um DataFrame pandas, o seguinte erro é levantado:

In [2]:
file = "data/raw/DadosBO_SP/DadosBO_2023_1(ROUBO DE CELULAR).xls"
df_test = pd.read_excel(file)

ValueError: Excel file format cannot be determined, you must specify an engine manually.

#### Solucionado os problemas de importação

A exceção `ValueError: Excel file format cannot be determined, you must specify an engine manually` indica que formato do arquivo não pode ser determinado automaticamente. Essa exceção geralmente é lançada quando a `pandas` não consegue identificar o formato do arquivo do Excel. O erro também sugere que precisamos especificar manualmente o mecanismo (ou "engine") que o `pandas` deve usar para ler o arquivo do Excel. No nosso caso, como se trata de um arquivo `xls`, um modelo mais antigo da Microsoft, vamos utilizar a biblioteca `xlrd`, que é o padrão para este tipo de situação.

In [3]:
!conda install xlrd -y

Solving environment: done

# All requested packages already installed.



Utilizando a engine `xlrd`, obtemos o seguinte erro:

In [4]:
df_test = pd.read_excel(file, engine="xlrd")

XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'A\x00N\x00O\x00_\x00'

A exceção `XLRDError: Unsupported format, or corrupt file: Expected BOF record; found b'A\x00N\x00O\x00_\x00'` ocorreu porque o `xldr` esperava encontrar um registro de início de arquivo ([BOF - Beginning of File](https://learn.microsoft.com/en-us/openspecs/office_file_formats/ms-xls/4d6a3d1e-d7c5-405f-bbae-d01e9cb79366)), mas encontrou algo diferente, representado como a sequência de bytes `b'A\x00N\x00O\x00_'`. Essa sequência de bytes pode ser uma pista para a natureza do problema, mas não é possível ter certeza sem mais contexto.

Ocorre que BOF é um termo usado para se referir ao registro inicial que marca o início de arquivos. É uma sequência específica de bytes que marca o começo de um arquivo e ajuda os programas a entenderem como interpretar e processar o conteúdo do arquivo, seja ele binário ou em outro formato. No nosso caso, o `xldr` esperava encontrar o BOF de um arquivo binário `xls` da Microsoft, mas encontrou outra sequência de bytes; o que pode indicar que o arquivo não é um `xls` mas um outro formato.

Uma forma de descobrir o formato do arquivo é identificar seu MIMETYPE. O tipo [MIME (Multipurpose Internet Mail Extensions)](https://en.wikipedia.org/wiki/MIME) é uma convenção para especificar o tipo de conteúdo de um arquivo com base em sua natureza e formato. Embora o MIME tenha sido projetado principalmente para [SMTP (Simple Mail Transfer Protocol)](https://pt.wikipedia.org/wiki/Simple_Mail_Transfer_Protocol), seus tipos de conteúdo também são importantes em outros protocolos de comunicação, como o [HTTP (HyperText Transfer Protocol)](https://pt.wikipedia.org/wiki/Hypertext_Transfer_Protocol), por exemplo. No HTTP para a web, os servidores inserem um campo de cabeçalho MIME no início de qualquer transmissão. Os clientes usam o tipo de conteúdo ou o cabeçalho do tipo de mídia para selecionar um aplicativo visualizador apropriado para o tipo de dados indicado.  

Antes de tentar abrir o arquivo em um aplicativo externo, vamos tentar utilizar o comando [`file`](https://manned.org/file.1) do UNIX passando a opção `-i` para tentar obter uma descrição detalhada e precisa do formato incluindo o tipo MIME: 


In [2]:
!file -i ./data/raw/DadosBO_SP/DadosBO_2023_1\(ROUBO\ DE\ CELULAR\).xls

./data/raw/DadosBO_SP/DadosBO_2023_1(ROUBO DE CELULAR).xls: application/octet-stream; charset=binary


A saída não é animadora: `application/octet-stream; charset=binary` indica que o arquivo foi identificado como um fluxo de bytes genérico (application/octet-stream) e que não foi possível determinar um charset (conjunto de caracteres) específico. Em outras palavras, utilitário `file` não conseguiu identificar com precisão o formato específico do arquivo e o tratou como um fluxo genérico de bytes. A identificação específica do formato pode ser desafiadora quando não há informações claras nos primeiros bytes do arquivo ou quando o formato é ambíguo.

Por esta razão, a alternativa mais rápida é tentar executá-lo em outra aplicação e observar como este aquivo é lido. As imagens abaixo mostram minha tentativa com LibreOffice:


![pop-up](./img/2023-08-28_00-49.png)
![pop-up](./img/2023-08-28_00-54.png)

Note que, ao tentar abrir o arquivo com LibreOffice Calc, este é lido como um `csv` e que o charset é "UTF-16". Na caixa de pré-visualização dos campos, da primeira imagem, vemos que os dados possuem uma `→` indicando que o separador é uma tabulação. A segunda imagem mostra que, ao selecionar "Tabulação" nas "Opções de separadores", os dados se organizam. Isso significa que, como desconfiávamos, se trata de outro tipo de formato. Não é novidade que tenhamos que lidar com situações desse tipo quando se trata de dados públicos governamentais. 

A próxima questão que queremos responder é: podemos carregar o conjunto de dados em um `DataFrame` do `pandas` usando o método `read_csv()` em vez de `read_excel()`? 

In [5]:
df_test = pd.read_csv(file, sep="\t", encoding="UTF-16")

UnicodeError: UTF-16 stream does not start with BOM

Sim, aparentemente é possível, contudo, obtemos um `UnicodeError` que indica um problema no conjunto de caracteres. Tentaremos compreender e solucionar esta exceção em seguida.

#### Solucionando o problemas do conjunto de caracteres

A exceção `"UnicodeError: UTF-16 stream does not start with BOM` ocorre quando tentamos decodificar um fluxo de bytes, mas o fluxo de bytes não começa com o [BOM (Byte Order Mark)](https://en.wikipedia.org/wiki/Byte_order_mark) necessário. Em outras palavras, o decodificador do `pandas` esperava encontrar o BOM no início do fluxo de bytes UTF-16, mas não o encontrou. Isso geralmente ocorre quando o BOM foi omitido, ou quando o arquivo foi codificado em um formato diferente, mas erroneamente rotulado como UTF-16. Dado o que vimos até aqui, esta última possibilidade é bem plausível. Mas vamos continuar buscando pistas.

Para descobrir o esquema de codificação dos caracteres dos arquivos, vamos usar o utilitário [`enca`](https://linux.die.net/man/1/enca) do UNIX. O `enca` é um utilitário que detecta o conjunto de caracteres e a codificação de arquivos de texto e, também, pode convertê-los em outras codificações usando um conversor embutido ou bibliotecas externas como [libiconv](https://www.gnu.org/software/libiconv/), [librecode](https://ubuntu.pkgs.org/20.04/ubuntu-main-amd64/librecode-dev_3.6-24_amd64.deb.html) ou [cstocs](https://www.venea.net/man/Cz::Cstocs(3pm)).

Como meu objetivo é saber apenas o encoding, uso o seguinte comando passando um dos arquivos de dados:

In [3]:
!enca -L none ./data/raw/DadosBO_SP/DadosBO_2023_1\(ROUBO\ DE\ CELULAR\).xls

Universal character set 2 bytes; UCS-2; BMP
  LF line terminators
  Byte order reversed in pairs (1,2 -> 2,1)


#### Compreendendo o formato de codificação de caracteres do conjunto de dados

A saída indica que o encoding é Universal Character Set de 2 bytes, ou UCS-2. Esse é um formato de codificação de caracteres que representa cada caractere em 2 bytes (16 bits). Os outros elementos da saída são os seguintes:

- **BMP** refere-se ao "[Basic Multilingual Plane](https://en.wikipedia.org/wiki/Plane_(Unicode)#Basic_Multilingual_Plane)", que é uma parte do conjunto de caracteres Unicode que abrange a maioria dos caracteres comumente usados em várias línguas. 
- **LF line terminators**, indica que o arquivo usa terminadores de linha **LF (Line Feed)**, que é um caractere de controle para indicar o fim de uma linha de texto. Isso é comum em sistemas baseados em Unix/Linux e é representado pelos caracteres `\n`. 
- **Byte order reversed in pairs (1,2 -> 2,1)** significa que a ordem dos bytes é invertida em pares (1, 2 -> 2, 1). 

Como dito, em UCS-2, cada caractere é representado por dois bytes. A ordem normal de armazenamento desses bytes na memoria seria primeiro o byte de ordem baixa (**Low Order Byte**) e depois o byte de ordem alta (**High Order Byte**). No entanto, com a ordem invertida em pares, os bytes são armazenados de forma reversa, ou seja, o byte de ordem alta vem antes do byte de ordem baixa.

Os conceitos de "Low Order Byte" (LOB) e "High Order Byte" (HOB) estão relacionados à forma como os bytes são organizados na memória em sistemas que utilizam múltiplos bytes para representar informações , como conjuntos de caracteres ou valores numéricos. Eles estão ligados a um conceito mais amplo conhecido como [Codificação de Largura Variável](https://pt.wikipedia.org/wiki/Codifica%C3%A7%C3%A3o_de_largura_vari%C3%A1vel), onde os mais comuns são as codificações multibyte, que usam vários números de bytes (octetos, daí a saída genérica "application/octet-stream" do comando `file`) para codificar diferentes caracteres. 

O LOB é o byte menos significativo em uma sequência de bytes que compõem uma unidade de informação. Em uma codificação de caracteres ou valor numérico, o LOB geralmente carrega a parte menos significativa da informação. Em contrapartida, o HOB é o byte mais significativo em uma sequência de bytes. Ele carrega a parte mais significativa da informação.

"Menos significativa" e "mais significativa" são terminologias relacionadas com a importância relativa de partes individuais de um valor binário (sequência de bits) ao representar informações. Só pra relembrar: 
- **Bit**: A menor unidade de informação em um sistema binário, podendo ser 0 ou 1.
- **Byte**: Um conjunto de 8 bits.
- **Sequência de Bytes**: Valores numéricos, caracteres e outros tipos de dados frequentemente representados por sequências de bytes.

Quando falamos sobre "menos significativa" e "mais significativa", estamos nos referindo à posição desses bits e bytes na representação binária de um valor. Essa terminologia se origina do sistema posicional que usamos para representar números, incluindo números decimais e binários.

**Exemplo para ficar menos abstrato:**

Em um sistema posicional, como o sistema decimal que usamos cotidianamente, o valor de um dígito depende de sua posição. Por exemplo, no número decimal "314", o "3" na posição das centenas representa uma quantidade significativamente maior do que o "1" na posição das dezenas, que é, por sua vez, mais significativo do que o "4" na posição das unidades.

Em uma representação binária funciona da mesma forma: cada bit, em uma posição específica, tem um valor que é uma potência de 2. Por exemplo, no número binário "10110", o bit mais à esquerda (o "1" mais significativo) representa 16 (ou $2^4$), enquanto o bit mais à direita (o "0" menos significativo) representa 1 (ou $2^0$).

Ao falar sobre partes "mais significativas" e "menos significativas", estamos observando como as posições dos dígitos ou bits contribuem para o valor geral de um número ou caractere. As partes mais significativas têm um impacto maior no valor total, enquanto as partes menos significativas têm um impacto menor.

Portanto, se trata de uma maneira de descrever a importância relativa das posições de dígitos ou bits em uma representação numérica ou binária, com base na forma como o sistema posicional funciona. Para ilustrar, imagine que estamos representando o número decimal 314 em uma codificação numérica de 16 bits:

```
00000001 00111010
```
Temos 16 bits divididos em dois bytes. A representação binária desse número é "0000000100111010". Agora, analisemos essa representação em termos de "menos significativa" e "mais significativa":

**Menos Significativa**: Os bits à direita são considerados menos significativos. No nosso exemplo, "00111010" é a parte menos significativa. Alterações nesses bits teriam um impacto menor no valor geral do número.

**Mais Significativa**: Os bits à esquerda são considerados mais significativos. No nosso exemplo, "00000001" é a parte mais significativa. Alterações nesses bits teriam um impacto maior no valor geral do número.

A saída do `enca` informa que a ordem dessas partes é invertida. Isso ocorre, pois a ordem em que os LOBs e HOBs são organizados pode variar entre diferentes sistemas e arquiteturas de computador. Existem dois principais padrões de ordenação:

**Little Endian**: Nesse padrão, o LOB é armazenado antes do HOB. Ou seja, o byte de menor valor é armazenado primeiro na memória. Isso é comum em muitos sistemas, incluindo os de arquitetura x86 e x86-64, que são amplamente usados em PCs e laptops.

**Big Endian**: Nesse padrão, o HOB é armazenado antes do LOB. O byte de maior valor é armazenado primeiro na memória. Isso é comum em algumas arquiteturas de processadores, como PowerPC e algumas implementações de redes.

Esses conceitos são particularmente relevantes ao lidar com codificações de caracteres em que cada caractere é representado por múltiplos bytes (multibyte). A ordem dos bytes pode afetar a forma como os caracteres são interpretados e, portanto, é importante considerar a ordem de bytes correta ao lidar com diferentes sistemas e formatos de dados. É justamente que estamos enfrentando no momento.  

#### Importando os dados com a codificação de caracteres correta

O UCS-2 é um esquema de codificação mais antigo e mais restrito porque só pode representar os caracteres do BMP, isto é, os caracteres Unicode que podem ser representados em 16 bits. Caracteres fora do BMP não podem ser representados diretamente em UCS-2. Verifiquei na  [documentação do `pandas`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) que a [lista de encodings suportados pelo Python](https://docs.python.org/3/library/codecs.html#standard-encodings) não inclui UCS-2. Por tanto, este é o motivo do `pandas`, não suportar esse encoding.

Contudo, o esquema **UTF-16**, uma codificação mais moderna e flexível que pode representar todos os caracteres Unicode, incluindo aqueles fora do BMP, também representa caracteres em 2 bytes e é suportada pela linguagem. Por que o `pandas` não foi capaz de carregar os dados mesmo assim? A resposta é simples: precisamos informar o decodificador do `pandas` que a ordem dos bytes está invertida, ou seja, precisamos indicar o padrão **Little Endian**, ou **"LE"**, para que a biblioteca consiga ler os bytes na ordem correta: 

In [10]:
df_test = pd.read_csv(file, sep="\t", encoding="UTF-16 LE")

In [11]:
df_test.head()


,ANO_BO,NUM_BO,NUMERO_BOLETIM,BO_INICIADO,BO_EMITIDO,DATAOCORRENCIA,HORAOCORRENCIA,PERIDOOCORRENCIA,DATACOMUNICACAO,DATAELABORACAO,...,PLACA_VEICULO,UF_VEICULO,CIDADE_VEICULO,DESCR_COR_VEICULO,DESCR_MARCA_VEICULO,ANO_FABRICACAO,ANO_MODELO,DESCR_TIPO_VEICULO,QUANT_CELULAR,MARCA_CELULAR
0,2023,2059,2059/2023,01/01/2023 00:08:34,01/01/2023 00:08:34,29/12/2022,NaN,A NOITE,30/12/2022,01/01/2023 00:08:34,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Xiaomi
1,2023,27,27/2023,01/01/2023 00:39:51,01/01/2023 00:39:51,31/12/2022,23:32,A NOITE,01/01/2023,01/01/2023 00:39:51,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Apple
2,2023,8583,8583/2023,01/01/2023 00:47:10,01/01/2023 00:47:12,30/01/2022,23:20,A NOITE,31/12/2022,01/01/2023 00:47:10,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Apple
3,2023,8584,8584/2023,01/01/2023 00:47:35,01/01/2023 00:47:38,30/12/2022,22:05,A NOITE,31/12/2022,01/01/2023 00:47:35,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Apple
4,2023,8588,8588/2023,01/01/2023 00:48:47,01/01/2023 00:48:49,30/12/2022,20:30,A NOITE,31/12/2022,01/01/2023 00:48:47,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,1.0,Xiaomi


### Padronizando o nome dos arquivos e importando os dados no Pandas

Agora que desvendamos a natureza dos arquivos de dados fornecidos pelo governo, vamos realizar as tarefas de transformar o nome dos arquivos para um padrão seguro e, finalmente, importar os datasets como o encoding correto para gerar um `DataFrame` e prosseguir para a etapa: [Processamento dos Dados]("./04_processamento_dos_dados.ipynb"),  que tornará os dados úteis para análise exploratória. 

O script a seguir, busca resolver ambos os problemas, padronizando o nome dos arquivos para "DadosBO_{ano}_{nome_do_mês}, e realizando a leitura correta dos bytes gerando um DataFrame no qual podemos começar a trabalhar:

In [9]:
import os
import shutil
import re

In [12]:
input_dir = "data/raw/DadosBO_SP/"
output_dir = "data/processed/DadosBO_SP/"

os.makedirs(output_dir, exist_ok=True)

MONTHS = {
    "1": "Janeiro",
    "2": "Fevereiro",
    "3": "Março",
    "4": "Abril",
    "5": "Maio",
    "6": "Junho",
    "7": "Julho",
    "8": "Agosto",
    "9": "Setembro",
    "10": "Outubro",
    "11": "Novembro",
    "12": "Dezembro"
}

pattern = re.compile(r"(\d+)\(ROUBO DE CELULAR\)")


def process_file(input_dir, output_dir):
    for filename in os.listdir(input_dir):

        if filename.endswith(".xls"):
            match = pattern.search(filename)

            if match:
                num_month = match.group(1)
                name_month = MONTHS.get(num_month, num_month)

                new_name = pattern.sub(f"{name_month}", filename)

                old_path = os.path.join(input_dir, filename)
                new_path = os.path.join(output_dir, new_name)

                shutil.copy(old_path, new_path)

                yield new_path


file_path_generator = process_file(input_dir, output_dir)

datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")
            for file in file_path_generator]

df = pd.concat(datasets, axis=0, ignore_index=True)

pd.set_option("display.max_columns", None)


/tmp/ipykernel_728800/2381841722.py:46: DtypeWarning: Columns (31,32,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")
/tmp/ipykernel_728800/2381841722.py:46: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")
/tmp/ipykernel_728800/2381841722.py:46: DtypeWarning: Columns (31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  datasets = [pd.read_csv(file, sep='\t', encoding="UTF-16 LE")


In [13]:
df.shape

(119158, 54)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119158 entries, 0 to 119157
Data columns (total 54 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ANO_BO                    119158 non-null  int64  
 1   NUM_BO                    119158 non-null  int64  
 2   NUMERO_BOLETIM            119158 non-null  object 
 3   BO_INICIADO               119158 non-null  object 
 4   BO_EMITIDO                119158 non-null  object 
 5   DATAOCORRENCIA            119158 non-null  object 
 6   HORAOCORRENCIA            110412 non-null  object 
 7   PERIDOOCORRENCIA          119158 non-null  object 
 8   DATACOMUNICACAO           119158 non-null  object 
 9   DATAELABORACAO            119158 non-null  object 
 10  BO_AUTORIA                119158 non-null  object 
 11  FLAGRANTE                 119158 non-null  object 
 12  NUMERO_BOLETIM_PRINCIPAL  37625 non-null   object 
 13  LOGRADOURO                111762 non-null  o

et voilà!

---

## Referências
- [Microsoft - BOF (Beginning of File)](https://learn.microsoft.com/en-us/openspecs/office_file_formats/ms-xls/4d6a3d1e-d7c5-405f-bbae-d01e9cb79366)
- [Wikipedia - BOM (Byte Order Mark)](https://en.wikipedia.org/wiki/Byte_order_mark)
- [Wikipedia - BPM (Basic Multilingual Plane)](https://en.wikipedia.org/wiki/Plane_(Unicode)#Basic_Multilingual_Plane)
- [Wikipedia - MIME (Multipurpose Internet Mail Extensions)](https://en.wikipedia.org/wiki/MIME)
- [Wikipedia - Codificação de Largura Variável](https://pt.wikipedia.org/wiki/Codifica%C3%A7%C3%A3o_de_largura_vari%C3%A1vel)
- [Unicode Consortium - Glossary](https://www.unicode.org/glossary/)
- [UTF-8 and Unicode FAQ for Unix/Linux](https://www.cl.cam.ac.uk/~mgk25/unicode.html)
- [Wikipedia - Universal Character Set](https://en.wikipedia.org/wiki/Universal_Coded_Character_Set)
- [Wikipedia - UTF-16](https://en.wikipedia.org/wiki/UTF-16)
- [Wikipedia - Endianness](https://en.wikipedia.org/wiki/Endianness)
- [Documentação Pandas](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)
- [Documentação Python](https://docs.python.org/3/library/codecs.html#standard-encodings)